# KAPy Output Visualisation

In [1]:
#First set the working directory correctly. Everything is KAPy is relative to the project / repo root directly. Ideally, this could be handled via
#the options to run the jupyter notebook, but that doesn't seem to be possible. Instead we force the wd relative to this file
import os
os.chdir("../../")
#Also need to make sure that KAPy can be read from the workflow directory
import sys
sys.path.append('workflow')

In [2]:
#Import other libraries
import KAPy
import IPython.display 
import pandas as pd
import os
import yaml

In [3]:
#Load configuration and workflow
#We assert that the config file has been validated and is ok. This
#may not hold, however, if json-schema defaults are being used - a good
#reason to avoid this behaviour
config=KAPy.loadConfig()
#Now get the workflow
wf=KAPy.getWorkflow(config)


In [23]:
#Get the list of potential inputs that can be used out of the workflow. Specifically, this is the ensstats netcdf files, and the areal stats.
#We don't consider other combinations at the moment
def makeInputDict(d):
    inpTbl=pd.DataFrame(list(d.keys()),columns=['path'])
    inpTbl['fname']=[os.path.basename(f) for f in inpTbl['path']]
    inpTbl['indId']=inpTbl['fname'].str.extract("^(.*?)_.*$")
    inpTbl['indSrc']=inpTbl['fname'].str.extract("^.*?_(.*?)_.*$")
    inpTbl['indScen']=inpTbl['fname'].str.extract("^.*?_.*?_(.*?)_.*$")
    inpDict=inpTbl.groupby('indId').apply(lambda x:list(x['path']),include_groups=False).to_dict()
    return(inpDict)

asDict=makeInputDict(wf['arealstats'])
ensDict=makeInputDict(wf['ensstats'])

In [8]:
#Loop over available indicators to make plots
for thisInd in config['indicators'].values():
    #Print the indicator name as a header
    IPython.display.display(IPython.display.Markdown(f"## Indicator {thisInd['id']} - {thisInd['name']}"))
   
    #But what should we plot? It depends on the nature of the indicator
    # * Period-based indicators should plot the spatial map and the plots
    # * Yearly (or monthly) based indicators show a time series
    if thisInd['time_binning']=="periods":
        #Box plot
        bxpFiles=asDict[str(thisInd['id'])]
        p=KAPy.makeBoxplot(config,thisInd,bxpFiles)
        p.show()

        #Spatial plot
        spFiles=ensDict[str(thisInd['id'])]
        p=KAPy.makeSpatialplot(config,thisInd,spFiles)
        p.show()
    
    elif thisInd['time_binning'] in ["years",'months']:
        #Time series plot
        lpFiles=asDict[str(thisInd['id'])]
        p=KAPy.makeLineplot(config,thisInd,lpFiles)
        p.show()


## Indicator 101 - Annual mean temperature

KeyError: '101'